In [ ]:
import pandas
from main import get_place_data as GetPlaceData
from main.utils.data_logger import Logger
from main.core.data_handler import DataHandler
from main.core.classifier import Recommender
from main import make_recommendation
import main.utils as utils
data_handler = DataHandler()

%reload_ext sql
%sql mysql+pymysql://root:secret@db:3306/my_db

### Getting data from google places API

In [ ]:
my_logger = Logger(logger_name='Test_run').get_logger()
my_logger.info('Start')
results = await GetPlaceData.run(query='Sao Paulo, SP', type='restaurant',radius=5000, resolution=50, timeout=20)
my_logger.info('Done!')

### Preprocessing data to put it in to the db.

In [ ]:
data_frame = data_handler.dict_list_to_dataframe(results)
data_frame['name'] = data_frame['name'].apply(lambda name: emoji.demojize(name))
data_frame['name'] = data_frame['name'].apply(lambda name: strip_accents(name))
data_frame['formatted_address'] = data_frame['formatted_address'].apply(lambda name: strip_accents(name))
data_frame['lat'] = data_frame['geometry'].apply(lambda row: row['location']['lat'])
data_frame['lon'] = data_frame['geometry'].apply(lambda row: row['location']['lng'])
data_frame.columns

### Connecting to db, filtering revelant info and adding it to db.

In [ ]:
engine = data_handler.db_connect(db_url = "mysql+pymysql://root:secret@db:3306/my_db")

In [ ]:
data_frame2 = data_frame[['business_status', 'name', 'place_id', 'price_level','lat', 'lon' ,'rating','user_ratings_total']]
data_handler.add_data_to_db(data_frame2, name='RESTAURANT_SP_CAP2',con = engine, if_exists='replace', index=False, method='multi')


#### Now quering this data from db, making it to a dataframe, and dropping NaN values.

In [ ]:
data_frame = data_handler.get_data_from_db(query="SELECT * FROM RESTAURANT_SP_CAP_2", con=engine)
data_frame.dropna(inplace=True)
data_frame['place_id']=data_frame['place_id'].astype(str)
data_frame['name']=data_frame['name'].astype(str)
#data_frame['formatted_address']=data_frame['formatted_address'].astype(str)
print(data_frame.shape)
data_frame.dtypes

### Getting only relevant data to classifier

In [ ]:
data_frame.drop_duplicates(['place_id'], keep='last', inplace=True)
data_frame.reset_index(drop=True, inplace=True)
data_frame2=data_frame[['price_level', 'rating', 'user_ratings_total', 'lat', 'lon']]
data_frame2.shape




### Acessing the best k number through elbow method, using Within Cluster Sum of Squared Errors

In [ ]:
my_classifier = Recommender()
my_classifier.preprocess(data_frame2, standarize=True)
my_classifier.get_best_cluster_number(n_clusters=20,method='ms',init = "k-means++", n_init=12)


### Discovered that the best k is 4.

### Visualising clusters!

In [ ]:
my_classifier.train(n_clusters=4, init = "k-means++", n_init=12)
my_classifier.plot_intercluster_distance()

### Recommend locations.

In [ ]:
import pandas
restaurant = {'price_level':[1.0],
              'rating':[4.3],
              'user_ratings_total':[107],
              'lat ':[-24.128515],
              'lon':[-46.689152]}
restaurant = pandas.DataFrame(restaurant)
restaurant

In [ ]:
make_recommendation.make_recommendation(restaurant)